# Opti planning Galderma

import

In [43]:
import pandas as pd
import numpy as np
import csv
from pandas.tseries.offsets import BDay



Readfile and clean it

In [44]:
df = pd.read_excel("data.xlsx")
#sort by date fabrication
df.sort_values(by='DATE FABRICATION', inplace=True, ascending=True)
#remove unnamed columns
df = df.drop(df.filter(regex='Unnamed').columns, axis=1)
#remove rows with missing values
df = df.dropna()
df
df.to_csv('data/original_data.csv', index=False)

get all matiere need by day 

In [45]:


df.set_index('DATE FABRICATION', inplace=True)

df_groupby = df.groupby(['MP CODE', df.index.date]).agg({'BESOIN (KG)': 'sum', 'Matiere_OF': lambda x: list(x)})
#remove rows with 0.000 in BESOIN (KG)
df_groupby = df_groupby.reset_index()

df_groupby = df_groupby[df_groupby['BESOIN (KG)'] != 0.000]
#add column NB PESEE
df_groupby['NB PESEE'] = df_groupby['Matiere_OF'].apply(lambda x: len(set(x)))
df_groupby = df_groupby.reset_index()
#drop index column double
df_groupby = df_groupby.drop(['index'], axis=1)
#rename date column
df_groupby.rename(columns={'level_1': 'DATE FABRICATION'}, inplace=True)


df_groupby


,MP CODE,DATE FABRICATION,BESOIN (KG),Matiere_OF,NB PESEE
0,C20100,2024-04-16,0.8151,[C201000000456352],1
1,C20100,2024-04-18,0.8151,[C201000000456351],1
2,C20300,2024-03-11,161.6000,[C203000000455108],1
3,C20300,2024-03-14,40.8000,[C203000000455031],1
4,C20300,2024-03-15,40.8000,[C203000000455033],1
...,...,...,...,...,...
1595,C3902569,2024-04-15,153.0000,[C39025690000456500],1
1596,C3902569,2024-04-16,153.0000,[C39025690000456499],1
1597,C3902569,2024-04-18,153.0000,[C39025690000456504],1
1598,C3902569,2024-04-19,153.0000,[C39025690000456505],1


Get the date need to be create at last (48h before (business day))

In [46]:
#BDay is a function that returns the business day offset by the given number of business days
df_groupby['DATE_48h_avant'] = df_groupby['DATE FABRICATION'] - BDay(2)
df_groupby.sort_values('DATE_48h_avant', inplace=True)
df_groupby.to_csv('data/final_clean_data.csv', index=False)
df_groupby 

,MP CODE,DATE FABRICATION,BESOIN (KG),Matiere_OF,NB PESEE,DATE_48h_avant
925,C27020,2024-03-05,5.0500,[C270200000455182],1,2024-03-01 00:00:00
1267,C28750,2024-03-05,227.2448,[C287500000453897],1,2024-03-01 00:00:00
1265,C28710,2024-03-05,3.2844,"[C287100000455116, C287100000455117]",2,2024-03-01 00:00:00
1264,C28700,2024-03-05,40.4000,[C287000000455182],1,2024-03-01 00:00:00
652,C23650,2024-03-05,29.4792,"[C236500000455120, C236500000455116, C23650000...",4,2024-03-01 00:00:00
...,...,...,...,...,...,...
79,C20543,2024-04-26,6.1200,[C205430000456251],1,2024-04-24 00:00:00
1001,C27050,2024-04-26,12.2400,[C270500000456251],1,2024-04-24 00:00:00
235,C21526,2024-04-26,67.3200,[C215260000456251],1,2024-04-24 00:00:00
495,C23150,2024-04-26,6.1200,[C231500000456251],1,2024-04-24 00:00:00


Usefull functions

In [47]:

#some  usefull functions

# def add_week(df, week_number, add_value):
#     df.loc[df['week_number'] == week_number, 'temps semaine'] += add_value
#     return df

# def add_day(df,day,add_value):
#     df.loc[df['date'] == day, 'temps jour'] += add_value
#     return df

def add_matiere(df,day,row,add_value,week_number):
    if row["MP CODE"] in df.loc[df['date'] == day, 'matieres'].values[0]:
        df.loc[df['week_number'] == week_number, 'temps semaine'] += add_value
        df.loc[df['date'] == day, 'temps jour'] += add_value
        return df
    else:
        df.loc[df['date'] == day, 'matieres'].values[0].append(row["MP CODE"])
        df.loc[df['week_number'] == week_number, 'temps semaine'] += add_value
        df.loc[df['date'] == day, 'temps jour'] += add_value
        return df

def update_answer_df(date,row,ans_df,original_df,old_time):    
    cpt = 0
    for of in row["Matiere_OF"]:
        original_row = original_df[original_df['Matiere_OF'] == of]
        if old_time*60 == 15:
            time = 15
        else:
            
            if cpt == 0:
                time = 30
            else:
                time = 15
        cpt+=1
        add_row = {
                        'MP_OF': original_row['Matiere_OF'].values[0],
                        'MP CODE': original_row['MP CODE'].values[0],
                        'MP NAME': original_row['MP NAME'].values[0],
                        'BESOIN (KG)': original_row['BESOIN (KG)'].values[0],
                        'ORDRE PRODUCTION': original_row['ORDRE PRODUCTION'].values[0],
                        'DATE FABRICATION': original_row.index[0],
                        'DATE PESEE': date,
                        'Temps pesee (15 ou 30 minutes)': time
                        }
        ans_df = pd.concat([ans_df, pd.DataFrame([add_row])], ignore_index=True)
    return ans_df


def clean_file():

    with open('data/timetable.csv', 'w') as file:
        pass 
    with open('data/evolution.csv', 'w') as file:
        pass 

    with open('data/time_info.csv', 'w') as file:
        pass
    
    with open('data/days.csv','w') as file:
        pass
    with open('data/remains.csv','w') as file:
        pass
    
    


Algo

In [48]:
WEEK_LIMITE = 150 #hours
DAY_LIMITE = 30 #hours 
DEFAULT_TIME = 0.5 #hours
ADD_TIME = 0.25 #hours


original_df = df.copy()
first_date  = df_groupby['DATE_48h_avant'].min()
last_date = df_groupby['DATE_48h_avant'].max()
dates = pd.date_range(start=first_date, end=last_date, freq='B')
#time dataframe in order to check limite
time_df = pd.DataFrame(index=dates, columns=['temps jour', 'temps semaine'])
time_df.reset_index(inplace=True)
time_df.rename(columns={'index': 'date'}, inplace=True)
time_df['temps jour'] = 0
time_df['temps semaine'] = 0
time_df['week_number'] = time_df['date'].dt.isocalendar().week
time_df['matieres'] = [[] for _ in range(len(time_df))]
remains_df = pd.DataFrame()
#our data df order by date
df_final = df_groupby.copy()
incidents_df = pd.DataFrame(columns=df_final.columns)

ans_df = pd.DataFrame(columns=['MP_OF', 'MP CODE', 'MP NAME', 'BESOIN (KG)', 
                               'ORDRE PRODUCTION', 'DATE FABRICATION', 'DATE PESEE', 'Temps pesee (15 ou 30 minutes)'])

clean_file()

for final_index, row in df_final.iterrows():
    is_inserted = False
    #get the date of the row
    date = row['DATE_48h_avant']
    #get the week number
    week_number = date.week
    
    #Case where their is still place in days before so we can us them
    max_time = (row['NB PESEE']-1) * ADD_TIME + DEFAULT_TIME
    
    #insert in the minimum row
    while not is_inserted:
        days = time_df[time_df['date'] < date]
        days = days[days['temps semaine'] + max_time< WEEK_LIMITE]
        days = days[days['temps jour'] + max_time < DAY_LIMITE]
    
        if days.shape[0] > 0:
            min_time_day_index = days['temps jour'].idxmin()
            days_row = days.loc[min_time_day_index]
            print(days_row["temps jour"])
            days_date = days_row['date']
            days_week_number = days_date.week
        else:
            break
            
        if row['MP CODE'] in days_row['matieres']:
            days_time = row["NB PESEE"] * ADD_TIME
        else: 
            days_time = (row["NB PESEE"]-1) * ADD_TIME + DEFAULT_TIME
        if ((days_row['temps jour'] + days_time <= DAY_LIMITE)):
            #add to the timetable
            time_df = add_matiere(time_df,days_date,row,days_time,days_week_number)
            #update the answer dataframe
            ans_df = update_answer_df(days_date,row,ans_df,original_df,days_time)
            is_inserted = True
        else:
            # remain_time = DAY_LIMITE - time_df[time_df['date'] == days_date]['temps jour']
            remain_time = DAY_LIMITE - days_row['temps jour']
            if (remain_time > 0.5):
                print(f"remaining time for {days_date} is {remain_time} and we need {days_time}")
    
    
    
    
    

    #determine time to add
    if row['MP CODE'] in time_df[time_df['date'] == date]['matieres']:
        time = row["NB PESEE"] * ADD_TIME
    else:
        time = (row["NB PESEE"]-1) * ADD_TIME + DEFAULT_TIME
        
        
        
    #check if we still have time to add the row and if it's the last day available
    if ((time_df[time_df['date'] == date]['temps semaine'] + time <= WEEK_LIMITE).all()
        and (time_df[time_df['date'] == date]['temps jour'] + time <= DAY_LIMITE).all()
        and is_inserted == False):
        #add to the timetable
        time_df = add_matiere(time_df,date,row,time,week_number)
        #update the answer dataframe
        ans_df = update_answer_df(date,row,ans_df,original_df,time)
        is_inserted = True



                    
                    
                    
    #Case where day is full but it's last day available
    if(row["DATE_48h_avant"] == date and is_inserted == False):
        #edit the incidents dataframe
        incidents_df = pd.concat([incidents_df, pd.DataFrame(row).transpose()])
        #add in the time_info
        time_df = add_matiere(time_df,date,row,time,week_number)
        #add in the answer dataframe
        ans_df = update_answer_df(date,row,ans_df,original_df,time)
        is_inserted = True
        
        
        
    
    days = days[days["date"] != date]
    days.to_csv("data/days.csv")
    
    
    if (is_inserted == False):
        #case not insert at all ! Why ? 
        
        remains_df = pd.concat([remains_df, pd.DataFrame(row).transpose()])
        
    
min_time_day_index = days['temps jour'].idxmin()
days_row = days.loc[min_time_day_index]
        
#Save data
original_df.to_csv('data/evolution.csv', index=False)
ans_df.to_csv('data/timetable.csv', index=False)
time_df.to_csv('data/time_info.csv', index=False) 
remains_df.to_csv('data/remains.csv', index=False)
incidents_df.to_csv('data/incidents.csv', index=False)

#check if we have the same number of rows
print(original_df.shape[0])
print(ans_df.shape[0])
cpt = 0
for index,row in remains_df.iterrows():
    for case in row["Matiere_OF"]:
        cpt+=1
        
print(f"cpt : {cpt}") 
print(original_df.shape[0] == cpt + ans_df.shape[0])




C:\Users\mathy\AppData\Local\Temp\ipykernel_3344\956730661.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.5]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['week_number'] == week_number, 'temps semaine'] += add_value
C:\Users\mathy\AppData\Local\Temp\ipykernel_3344\956730661.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.5]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['date'] == day, 'temps jour'] += add_value
C:\Users\mathy\AppData\Local\Temp\ipykernel_3344\956730661.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old be

8.5
9.0
9.5
9.75
10.0
10.5
11.5
11.75
12.25
12.75
13.5
14.0
14.5
15.0
15.5
15.75
16.25
17.25
18.25
18.75
19.5
20.25
20.5
0.0
0.5
1.0
1.5
2.0
2.5
3.0
3.5
4.0
4.5
0.0
0.5
1.25
1.75
2.25
2.75
3.75
4.25
5.0
5.5
6.0
6.0
6.5
7.0
7.5
7.75
8.25
8.5
8.75
9.25
9.25
9.75
9.75
10.25
10.5
10.75
11.0
11.25
11.5
12.0
12.5
12.75
13.25
13.75
14.0
14.5
14.5
15.0
15.0
0.0
0.5
1.25
2.25
3.25
3.75
4.25
5.5
6.0
6.5
7.0
8.0
9.5
10.5
11.0
12.0
12.5
13.25
14.0
14.75
15.5
15.75
16.25
16.25
16.5
16.75
17.25
17.25
17.75
17.75
18.0
18.25
18.5
0.0
0.5
1.0
1.5
2.0
2.75
3.25
4.75
5.5
6.0
6.75
8.0
8.75
9.25
10.0
10.75
11.25
12.0
12.5
13.25
14.0
14.75
15.25
15.75
16.25
17.0
18.0
18.5
18.75
19.0
19.0
19.0
19.25
19.5
20.0
20.25
20.5
20.5
20.75
21.0
21.0
21.0
21.25
21.25
21.5
21.5
21.5
21.5
21.75
22.0
22.0
22.0
0.0
0.75
1.25
2.0
2.75
4.0
5.5
6.5
7.25
8.0
9.0
9.75
10.25
11.0
11.75
12.5
13.0
13.75
14.5
15.25
16.0
16.75
17.5
18.0
18.75
19.5
20.5
0.0
0.5
1.0
1.5
2.0
2.5
3.0
3.5
4.25
4.75
5.25
5.75
6.25
6.75
7.25
7.75
8.25
9.0